# Assignment 6
## Problem 1

#### part a
||0| 1 | 2 | 3 | 4 | 5 | 6 | 7 |
|---|---|---|---|---|---|---|---|---|
|A|<ins>0</ins>|0|0|0|0|0|0|0|
|B|$\infty$|<ins>1</ins>,A|1,A|1,A|1,A|1,A|1,A|1,A|
|C|$\infty$|$\infty$|<ins>3</ins>,B|3,B|3,B|3,B|3,B|3,B|
|D|$\infty$|$\infty$|$\infty$|<ins>4</ins>,C|4,C|4,C|4,C|4,C|
|E|$\infty$|4,A|4,A|4,A|<ins>4</ins>,A|4,A|4,A|4,A|4,A|4,A|4,A|
|F|$\infty$|8,A|7,B|7,B|7,B|6,E|<ins>6</ins>,E|6,E|
|G|$\infty$|$\infty$|7,B|5,C|5,C|<ins>5</ins>,C|5,C|5,C|
|H|$\infty$|$\infty$|$\infty$|$\infty$|8,D|6,E|6,E|<ins>6</ins>,E|

#### part b

## Problem 2
The general approach for the problem will be to first traverse the matrix, and represent it as an undirected graph in which each tile is a vertex, and there is an edge between vertices if those two vertices represent adjacent land tiles. Once we have this representation, we simply have to perform depth first search and keep track of how many distinct connected components of the graph there are. The graph will obvioulsy not necessarily be connected. Depth first search will explore all connected nodes given a start node. We simply need to count how many times this explore function is called before all nodes are explored.

In [163]:
# I coded it for some reason - not worth the effort tbh
class FindIslands:
    def __init__(self,M):
        # first convert the matrix M into an adjecency list
        # assumes M is a square
        self.M = M
        self.dim = len(M)
        self.adj_list = [[] for _ in range(self.dim**2)]
        self.build_graph()
        self.visited = {}
        print(self.dfs())
  
    # given square matrix M, (0 = water, land = 1) convert it into an adjacency list
    def build_graph(self):
        for v in range((self.dim)**2):
            if self.M[v//self.dim][v%self.dim] == 1: #check that current tile is land
                if (v%self.dim != self.dim-1) and self.M[v//self.dim][v%self.dim + 1] == 1: #check tile to the right of current
                    self.adj_list[v].append(v+1) #graph is undirected so add connection for both tiles
                    self.adj_list[v+1].append(v)
                if (v//self.dim != self.dim-1) and M[v//self.dim + 1][v%self.dim] == 1: #check tile underneath the current
                    self.adj_list[v].append(v+self.dim) #graph is undirected so add connection for both tiles
                    self.adj_list[v+self.dim].append(v)
        
    def explore(self,v):
        #mark node as visited
        self.visited[v] = 0
        for u in self.adj_list[v]:
            if u not in self.visited:
                self.explore(u)

    def dfs(self):
        # now that we have adjacency list perform DFS
        island_count = 0
        for v in range(self.dim**2):
            if v not in self.visited and self.M[v//self.dim][v%self.dim] == 1: #only need to explore a tile if it is a land tile and hasn't been explored already
                self.explore(v)
                island_count += 1
        return island_count
    

In [164]:
M = [
[0,1,1,0,0],
[1,1,0,1,1],
[1,0,1,0,1],
[0,1,0,0,1],
[1,1,0,0,0]    
] #example matrix
prob = FindIslands(M)

4


## Problem 3

Idea is to BFS until you find the destination square.

In [165]:
import numpy as np
def valid_moves(pos,n):
    moves = {'ul':np.array([-1,2]),
             'ur':np.array([1,2]),
             'ru':np.array([2,1]),
             'rd':np.array([2,-1]),
             'dl':np.array([-1,-2]),
             'dr':np.array([1,-2]),
             'lu':np.array([-2,1]),
             'ld':np.array([-2,-1])}
    for key in moves:
        moves[key] += np.array(pos)
        if moves[key][0] < 0 or moves[key][1] < 0 or moves[key][0] > n-1 or moves[key][1] > n-1:
            moves[key] = None
    return moves
def chess_moves(start,dest,n):
    dist_list = {start:(0,None)}
    queue = []
    queue.append(start)
    while len(queue) != 0:
        pos = queue.pop(0)
        moves = valid_moves(pos,n)
        for key in moves:
            try:
                if moves[key] == None:
                    continue
            except:
                move = tuple(moves[key])
                if move not in dist_list:
                    dist_list[move] = (dist_list[pos][0] + 1,pos,key)
                    queue.append(move)
                    if move == dest:
                        break
    
    move_sequence = []
    curr = dest
    while dist_list[curr][1] != None:
        move_sequence.insert(0,dist_list[curr][2])
        curr = dist_list[curr][1]
    return dist_list[dest][0],move_sequence



In [166]:
print(chess_moves((0,0),(7,7),8))


(6, ['ur', 'ul', 'ur', 'ru', 'rd', 'ru'])


## Problem 4

In [184]:
class PriorityQueue:
    def __init__(self,mode = 'max'):
        self.mode = mode
        self.collection = []
        self.size = 0
        self.data_locations = {}
    def __right(self,i):
        return 2*i+1
    def __left(self,i):
        return 2*i+2
    def __parent(self,i):
        return (i-1)//2
    def __sift_down(self,i):
        l, r = self.__left(i),self.__right(i)
        if l < len(self.collection) and self.collection[l][1] > self.collection[i][1]:
            largest = l
        else: largest = i
        if r < len(self.collection) and self.collection[r][1] > self.collection[largest][1]:
            largest = r
        if largest != i:
            self.data_locations[self.collection[i][0]], self.data_locations[self.collection[largest][0]]  = largest, i
            self.collection[i],self.collection[largest] = self.collection[largest],self.collection[i]
            self.__sift_down(largest)

    def __bubble_up(self,i):
        p = self.__parent(i)
        if p == -1:
            return
        if self.collection[p][1] < self.collection[i][1]:
            self.data_locations[self.collection[p][0]], self.data_locations[self.collection[i][0]]  = i, p
            self.collection[p],self.collection[i] = self.collection[i],self.collection[p]
            self.__bubble_up(p)
        else:
            return
        
    def enqueue(self,task,priority):
        if self.mode == 'min':
            priority *= -1
        self.collection.append((task,priority))
        self.data_locations[task] = self.size
        self.__bubble_up(self.size)
        self.size += 1

    def deque(self):
        removed = self.collection[0][0]
        self.collection[0] = self.collection[self.size-1]
        self.size -= 1
        self.__sift_down(0)
        return removed if self.mode == 'max' else removed * -1
    
    def get_max(self):
        return self.collection[0][0]
    
    def change_priority(self,task,k):
        if self.mode == 'min':
            k *= -1
        index = self.data_locations[task]
        old = self.collection[index][1]
        self.collection[index] = (task, k)
        if k > old:
            self.__bubble_up(index)
        elif k <= old:
            self.__sift_down(index)
    
    def __len__(self):
        return self.size
    
    def __contains__(self,task):
        return task in self.data_locations

            


In [187]:
def djikstras(G,s):
    

[(3, -1)]
[(1, 0), (3, -1)]
[(3, -1), (1, -4)]
[(3, -1), (1, -4), (4, -8)]
[(3, -1), (1, -4), (4, -8), (3, -9)]
[(3, -1), (1, -4), (4, -8), (3, -9)]


4

In [ ]:
G = {
    'A': {'B': 1, 'E': 4, 'F': 8},
    'B': {'C': 2, 'F': 6, 'G': 6},
    'C': {'D': 1, 'G': 2},
    'D': {'G': 1, 'H': 4},
    'E': {'F': 5},
    'F': {},
    'G': {'F': 1, 'H': 1},
    'H': {}
}